In [15]:
from dataloader import GcMTLDataset
import pandas as pd
import torch

label_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/gc/n-labels/all_label.csv'
df = pd.read_csv(label_path, header=None)
keep_same_psize = 1000
file_name = df.iloc[:, 0].values
file_label = df.iloc[:, 1].values
root = '/data/wsi/TCTGC2625-features/plip-fine'
num_classes = [5]
num_task = 0
fine_concat = True
dataset = GcMTLDataset(file_name, file_label,root, num_task, num_classes, fine_concat=fine_concat, keep_same_psize=keep_same_psize)
# dataset = GcMTLFineDataset(file_name, file_label,root, num_task, num_classes, fine_concat=fine_concat)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=0, pin_memory=True)
for i, (f, l, t, f) in enumerate(data_loader):
    print(l.shape)
    print(t)

torch.Size([9])
tensor([1, 1, 0, 1, 2, 2, 2, 2, 2])


In [18]:
import torch
import pandas as pd
import shutil, os
from tqdm import tqdm

fn = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
df = pd.read_csv(fn)
label_wsi_names = df['wsi_name'].tolist()
label_wsi_names = [i.replace('.pt', '') for i in label_wsi_names]
print(len(label_wsi_names))
wsi_path = '/data/wsi/TCTGC50k/TCTGC50k-volume1'
output_wsi_path = '/data/wsi/TCTGC50k/TCTGC50k-volume-deprecated13'
wsi_names = os.listdir(wsi_path)
diff = set(wsi_names) - set(label_wsi_names)
diff = list(diff)
len(diff)

9854


0

In [17]:
# for i in diff:
#     shutil.move(os.path.join(wsi_path, i), os.path.join(output_wsi_path, i))

In [2]:
import torch
from model import MIL_MTL

def count_parameters(model):
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    params /= 1024 ** 2
    return params

num_task = 3
num_classes=[1,5,3]
input_dim = 256
model2 = MIL_MTL(num_classes=num_classes, num_task=num_task, input_dim=input_dim)
print((count_parameters(model2)))

size = (5, 10, 256) # [B, N, C]
mask = torch.ones(size[0], size[1], size[2])
a = torch.randn(size)
task_id = torch.randint(0, num_task, (size[0],))
output = model2(a, task_id)
print(output.shape)
# print(output.shape)

0.19251346588134766
torch.Size([5, 9])


In [ ]:
from dataloader import GcMTLDataset



